In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import optim
from data_utils import *
from model_utils import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

n_cudas = torch.cuda.device_count()
print(f"gpu count: {n_cudas}")
for i in range(n_cudas):
    print(torch.cuda.get_device_name(i))

gpu count: 0


# Prepare Dataset

In [3]:
# create raw dataset
x_train, y_train, x_valid, y_valid = GenerateRandomDataset.generate_slr_dataset_v1(
    true_bias=3.0, 
    true_weight=4.0, 
    sample_size=1000, 
    train_perc=0.8, 
    set_seed=42
)

In [4]:
# into Pytorch's tensor
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

In [5]:
# builds datasets
train_dataset = CustomDataset(x_train_tensor, y_train_tensor)
valid_dataset = CustomDataset(x_train_tensor, y_train_tensor)

In [6]:
# builds dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

In [7]:
# retrieve mini-batches
# next(iter(train_loader))

# Define Model

In [8]:
# define model
class MyLinearRegressionNested(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [9]:
# create model instance
reg_model = MyLinearRegressionNested().to(device)

# Training Loop

## Stochastic Gradient Descent

In [10]:
num_epochs = 1000
lr = 0.1
optimizer = optim.SGD(params=reg_model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

perform_train_step = make_train_step_fn(model=reg_model, loss_func=loss_fn, optimizer=optimizer)

In [11]:
for i in range(num_epochs):
    # get loss value
    current_loss = perform_train_step(x=x_train_tensor, y=y_train_tensor)
    
    if i%100 == 0:
        print(f"Epoch: {i}, Loss: {current_loss:0.5f}")

Epoch: 0, Loss: 31.30195
Epoch: 100, Loss: 0.01342
Epoch: 200, Loss: 0.01006
Epoch: 300, Loss: 0.00980
Epoch: 400, Loss: 0.00978
Epoch: 500, Loss: 0.00978
Epoch: 600, Loss: 0.00978
Epoch: 700, Loss: 0.00978
Epoch: 800, Loss: 0.00978
Epoch: 900, Loss: 0.00978


## Mini-batch Gradient Descent

In [14]:
num_epochs = 20
lr = 0.1
optimizer = optim.SGD(params=reg_model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

perform_train_step = make_train_step_fn(model=reg_model, loss_func=loss_fn, optimizer=optimizer)

In [15]:
losses = []

for i in range(num_epochs):
    
    mini_batch_losses = []
    
    for x_batch, y_batch in train_loader:
        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # print(x_batch.dtype)
        
        mini_batch_loss = perform_train_step(x=x_batch, y=y_batch)
        
        # accumulate losses, biases and weights for a mini batch
        mini_batch_losses.append(mini_batch_loss)
        
    loss = np.mean(mini_batch_losses)
    losses.append(loss)
    
    print(f"Epoch: {i}, Loss: {loss:0.5f}")

Epoch: 0, Loss: 0.00989
Epoch: 1, Loss: 0.00989
Epoch: 2, Loss: 0.00987
Epoch: 3, Loss: 0.00984
Epoch: 4, Loss: 0.00990
Epoch: 5, Loss: 0.00995
Epoch: 6, Loss: 0.00983
Epoch: 7, Loss: 0.00988
Epoch: 8, Loss: 0.00992
Epoch: 9, Loss: 0.00992
Epoch: 10, Loss: 0.00992
Epoch: 11, Loss: 0.00993
Epoch: 12, Loss: 0.00989
Epoch: 13, Loss: 0.00989
Epoch: 14, Loss: 0.00986
Epoch: 15, Loss: 0.00991
Epoch: 16, Loss: 0.00993
Epoch: 17, Loss: 0.00985
Epoch: 18, Loss: 0.00982
Epoch: 19, Loss: 0.00992


In [16]:
# model's state dict
print(reg_model.state_dict())

OrderedDict({'linear.weight': tensor([[4.0179]]), 'linear.bias': tensor([3.0033])})
